# Deploy Logic App + ACI

In this notebook, we will perform the final deployment so that the whole architecture is on Azure.

### Recap of the architecture:
The architecture for this workflow is centered on two main components:
- The AKS cluster that will perform the scoring by pulling (references of) images off the queue
- The Logic App + ACI that will trigger the workflow and populate the queue

At this point, we already have an AKS cluster up and running. It is continuously polling out Service Bus queue to check if there are any incoming messages to process. 

The next step is to deploy our Logic App + ACI components so that the workflow can be triggered by new videos that appear in blob storage. Once a new video is found in blob, Logic App will trigger the creation of an ACI container that will run the video preprocessing script, the add-to-queue script, and finally the postprocessing script. 

As soon as new items are added to the queue (from ACI), our AKS cluster will start pulling those items off and start processing them.

### Steps in this notebook:
1. Create the Docker image that will run inside of our ACI container
2. Test the Docker image by runnning it locally. 
3. Deploy the Logic App. The Logic App will go live once deployed. 
4. Test the Logic App deployment. To do so, we upload a new video to blob, which will triger the entire workflow.

---

### Import packages and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import jinja2
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Deploy Logic App

![Logic Apps](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_logic_app.PNG)

The *logic* behind the Logic App deployment is shown above:
1. When a blob is added, begin the workflow.
2. Check the blob name. 
    - if the blob name ends with `.mp4`:
        - make a request to the AKS endpoint
    - otherwise:
        - terminate in cancellation


Set up credentials and resource management client to create ARM deployments.

In [3]:
from azure.mgmt.resource.resources.models import DeploymentMode
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials

# Create credentials and ARM client
credentials = ServicePrincipalCredentials(client_id=get_key(env_path, "SP_CLIENT"),
                                          secret=get_key(env_path, "SP_SECRET"),
                                          tenant=get_key(env_path, "SP_TENANT"))

resource_client = ResourceManagementClient(credentials, get_key(env_path, "SUBSCRIPTION_ID"))

Create the deployment for the Azure blob storage connector.

In [7]:
# Create an Azure Blob Storage connector
with open('template.blob_connector.json') as f:
    blob_con_template = json.load(f)

blob_con_params = {
    "location": {"value": get_key(env_path, "REGION")},
    "subscription_id": {"value": get_key(env_path, "SUBSCRIPTION_ID")}
}

blob_con_props = {
    'mode': DeploymentMode.incremental,
    'template': blob_con_template,
    'parameters': blob_con_params
}

resource_client.deployments.create_or_update(
    resource_group_name=get_key(env_path, "RESOURCE_GROUP"), 
    deployment_name=get_key(env_path, "LOGIC_APP"), 
    properties=blob_con_props
)

Create the deployment for the Logic App.

In [11]:
# Create Logic App
with open('template.logic_app.json') as f:
    logic_app_template = json.load(f)

logic_app_params = {
    "location": {"value": get_key(env_path, "REGION")},
    "resource_group": { "value": get_key(env_path, "RESOURCE_GROUP") },
    "name": {"value": get_key(env_path, "LOGIC_APP")},
    "subscription_id": {"value": get_key(env_path, "SUBSCRIPTION_ID")},
    "storage_container_name": {"value": get_key(env_path, "STORAGE_CONTAINER_NAME")},
    "url_endpoint": {"value": "http://{}:8080/process/".format(get_key(env_path, "AKS_EXTERNAL_IP"))}
}

logic_app_props = {
    'mode': DeploymentMode.incremental,
    'template': logic_app_template,
    'parameters': logic_app_params
}

resource_client.deployments.create_or_update(
    resource_group_name=get_key(env_path, "RESOURCE_GROUP"), 
    deployment_name=get_key(env_path, "LOGIC_APP"), 
    properties=logic_app_props
)

Once the Logic App is deployed, go into the Azure portal and the Azure blob connector to authenticate. 

When you open up up the Azure blob connector, it should look like this:

![azure_blob_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_blob_connector_auth.PNG)

For the connector, click on the orange bar at the top to authenticate.

Once authenticated, your Logic App should be all set up and ready to trigger the workflow.

### Trigger logic app by adding a new video to the Azure blob container

Re-upload the orangutan video to blob with a new name: `logic_app_test_orangutan.mp4`

In [12]:
new_video_name = "logic_app_test_orangutan.mp4"

Make a copy the old `orangutan.mp4` video but named with the `<new_video_name>`. 

In [14]:
!cp data/orangutan.mp4 data/{new_video_name}

Use `curl` to hit the endpoint of the kubernetes cluster we just deployed.

In [16]:
!curl {get_key(env_path, "AKS_EXTERNAL_IP")}":8080/process?video_name="{new_video_name}

Processing logic_app_test_orangutan.mp4 in background...


---

Continue to the next [notebook](/notebooks/06_clean_up.ipynb).